In [35]:
import pickle
import pandas as pd
import numpy as np
import re
import ast
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer

In [2]:
with open('recipe_recommendation_model.pkl', 'rb') as f:
    vocab = pickle.load(f)

df = pd.read_csv('recipes_processed1.csv')

In [3]:
df['Ingredient_Vector'] = df['Ingredient_Vector'].apply(ast.literal_eval)

In [37]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def vectorize_ingredients(ingredient_str, vocab):
    vector = [0] * len(vocab)
    ingredients = [ingredient.strip().lower() for ingredient in re.split(r',|\s', ingredient_str) if ingredient]
    lemmatized_ingredients = [lemmatizer.lemmatize(ingredient) for ingredient in ingredients]
    for ingredient in lemmatized_ingredients:
        if ingredient in vocab:
            vector[vocab.index(ingredient)] = 1
    return vector

In [63]:
def vectorize_ingredients(ingredient_str, vocab):
    """ Convert input ingredients string into a vector based on vocabulary. """
    vector = [0] * len(vocab)
    ingredients = [ingredient.strip().lower() for ingredient in re.split(r',|\s', ingredient_str) if ingredient]
    lemmatized_ingredients = [lemmatizer.lemmatize(ingredient) for ingredient in ingredients]
    print("Lemmatized Ingredients :", lemmatized_ingredients)
    for ingredient in ingredients:
        if ingredient in vocab:
            vector[vocab.index(ingredient)] = 1
    return vector

In [41]:
def recommend_recipes(user_vector, df, top_n=5):
    recipe_vectors = np.array(df['Ingredient_Vector'].tolist())
    user_vector = vectorize_ingredients(user_vector, vocab)
    user_vector = np.array(user_vector).reshape(1, -1)
    similarity_scores = cosine_similarity(user_vector, recipe_vectors)[0]
    
    recipe_similarity = list(enumerate(similarity_scores))
    top_indices = sorted(recipe_similarity, key=lambda x: x[1], reverse=True)[:top_n]
    
    results = []
    for index, score in top_indices:
        name = df.loc[index, 'Title']
        diff = df.loc[index, 'Difficulty']
        url = df.loc[index, 'Link']  
        time = df.loc[index, 'Total Cook Time'] 
        similarity_score = round(score,3)
        results.append({'name': name, 'Difficulty': diff, 'url': url, 'time':time, 'score':similarity_score})
    
    return results

In [71]:
sample_ingredients = "onion, garlic, tomato, ginger"

# Call the recommendation function
results = recommend_recipes(sample_ingredients, df, top_n=5)
# Print the input ingredients
print("Entered ingredients: ", sample_ingredients, "\n")
# Print the results
print("Top Recipe Recommendations:\n")
for i, recipe in enumerate(results, start=1):
    print(f"{i}. {recipe['name']}")
    print(f"   Difficulty: {recipe['Difficulty']}")
    print(f"   Total time to cook: {recipe['time']}")
    print(f"   URL: {recipe['url']}")
    print(f"   Score: {recipe['score']}\n")

Lemmatized Ingredients : ['onion', 'garlic', 'tomato', 'ginger']
Entered ingredients:  onion, garlic, tomato, ginger 

Top Recipe Recommendations:

1. Black Forest Chicken Recipe
   Difficulty: easy
   Total time to cook: 30 mins
   URL: https://food.ndtv.com/recipe-black-forest-chicken-956210
   Score: 0.632

2. Veg Hakka Noodles Recipe
   Difficulty: easy
   Total time to cook: 30 mins
   URL: https://food.ndtv.com/recipe-veg-hakka-noodles-952069
   Score: 0.535

3. Egg Pasta Recipe
   Difficulty: easy
   Total time to cook: 25 mins
   URL: https://food.ndtv.com/recipe-egg-pasta-955600
   Score: 0.516

4. Sooji Manchurian Recipe
   Difficulty: easy
   Total time to cook: 30 mins
   URL: https://food.ndtv.com/recipe-sooji-manchurian-956126
   Score: 0.5

5. Mango Avocado & Ginger Bruschetta Recipe
   Difficulty: easy
   Total time to cook: 25 mins
   URL: https://food.ndtv.com/recipe-mango-avocado-ginger-bruschetta-958600
   Score: 0.5

